# FCN 논문 리뷰
FCN = Fully Convolutional Networks for Segmentic Segmentation

Segmentic Segmentation 문제를 위해 제안된 딥러닝 모델

![이미지](https://miro.medium.com/max/700/1*xWh0LhoDzwyjcxXpnz9lIQ.png)
입력 이미지에 대해 픽셀단위로 배경 및 각 개체의 클래스를 분할하는 동시에 예측.

FNC은 Semantic Segmentation 모델을 위해 기존에 이미지 분류에서 우수한 성능을 보인 CNN기반 모델(AlexNet, VGG16, GoogLeNet)을 목적에 맞춰 변형시킨 것

이러한 [Image classification model] to [Semantic segmentation model]은 크게 다음의 세 과정을 표현할 수 있다:
    
- Convolutionalization
- Deconvolution (Upsampling)
- Skip architecture

### Convolutionalization
Convolutionalization(컨볼루션화)이라는 표현의 의미를 이해하기 위해서는 기존의 이미지 분류 모델들을 먼저 살펴볼 필요가 있다.

![이미지](https://miro.medium.com/max/700/1*8ckDGCdl3YQy1V7sJ0hPTg.png)

Image classification 모델들은 기본적으로 내부 구조와 관계없이 모델의 근본적인 목표를 위해 출력층이 Fully-connected(이하fc) layer로 구성되어 있다.

이러한 구성은 네트워크의 입력층에서 중간부분까지 ConvNet을 이용하여 영상의 특징들을 추출하고 해당 특징들을 출력층 부분에서 fc를 이용해 이미지를 분류하기 위함이다.

그런데, Semantic Segmentation 관점에서는 fc layer가 갖는 한계점이 있다.

1. 이미지의 위치정보가 사라진다.
![이미지](https://miro.medium.com/max/700/1*IZ04YiEdorka6zx-vAzf-g.png)
fully-connected layer 연산 이후 Receptive field 개념이 사라진다.



2. 입력 이미지 크기가 고정된다.
![이미지](https://miro.medium.com/max/700/1*b7GEG3MP6zVmwoeASMQ1AA.png)
Dense layer에 가중치 개수가 고정되어 있기 때문에 바로 앞 레이어의 Feature Map의 크기도 고정되며, 연쇄적으로 각 레이어의 Feature Map 크기와 Input Image 크기 역시 고정된다.

Segmentation의 목적은 원본 이미지의 각 픽셀에 대해 클래스를 구분하고 인스턴스 및 배경을 분할하는 것으로 위치 정보가 매우 중요하다.
![이미지](https://miro.medium.com/max/700/1*SbR08aYSbiDX2hadjdOsRw.png)
Segmentation은 입력 이미지의 위치 정보가 유지되어야 한다.

이러한 fc-layer의 한계를 보완하기 위해 모든 fc-layer를 Conv-layer로 대체하는 방법을 택하였다.
![이미지](https://miro.medium.com/max/700/1*x6mXFw0k5GuiLxhHjwqg-A.png)
Fully-connected layer는 입력의 모든 영역을 receptive field로 보는 필터의 Conv layer로 생각할 수 있다.

※ receptive field는 출력 레이어의 뉴런 하나에 영향을 미치는 입력 뉴런들의 공간 크기

VGG16을 예로 살펴보면, 다음과 같이 출력층 부분의 마지막 3 fc-layers를 모두 Conv-layers로 변경한다.
![이미지](https://miro.medium.com/max/700/1*ddf0sJZDitiqVCWdjijzCA.png)
Dense Layer에서 Conv Layer로 변환하는 방식은 다음과 같다.
![이미지](https://miro.medium.com/max/700/1*2IuHjzPjjGXtDU-eAUzxeA.png)
첫 번째 fully-connected layer를 (7x7x512) 4096 filter conv로 변경하면 가중치의 수가 유지된다.
![이미지](https://miro.medium.com/max/700/1*S90pbw1OE2yy688eaK30ZA.png)
마지막 fc-layer의 경우 채널 차원을 클래스 수에 맞춘 1x1 conv로 변환한다.
VGG16에서 다섯 번째 max-pooling(size: 2x2, stride: 2) 연산 후 Feature map의 크기는 7x7이 된다. (입력 이미지의 크기가 224x224인 경우)
![이미지](https://miro.medium.com/max/700/1*5LhYXkhBg6kwbR1zN5Eg8g.png)
Convolutionalization을 통해 출력 Feature map은 원본 이미지의 위치 정보를 내포할 수 있게 되었다.

그러나 Semantic degmentation의 최종 목적인 픽셀 단위 예측과 비교했을 때, FCN의 출력 Feature map은 너무 coarse(거친, 알맹이가 큰)하다.

따라서, Coarse map을 원본 이미지 크기에 가까운 Dense map으로 변환해 줄 필요가 있다. 적어도 input image size 1/32보다는 해상도가 높을 필요가 있다.

### Deconvolution

Coarse map에서 Dense map을 얻는 몇가지 방법이 있다:
- Interpolation
- Deconvolution
- Unpooling
- Shift and Stitch

물론 Pooling을 사용하지 않거나, Pooling의 stride를 줄임으로써 Feature map의 크기가 작아지는 것을 처음부터 피할 수도 있다.

그러나, 이 경우 필터가 더 세밀한 부분을 볼 수는 있지만 Receptive Field가 줄어들어 이미지의 컨텍스트를 놓치게 된다.

또한, Pooling의 중요한 역할 중 하나는 특징맵의 크기를 줄임으로써 학습 파라미터의 수를 감소기키는 것인데, 
이러한 과정이 사라지면 파라미터의 수가 급격히 증가하고 이로인해 더 많은 학습시간을 요구하게 된다.

따라서, Pooling의 중요한 역할 중 하나는 특징맵의 크기를 줄임으로써 학습 파라미터의 수를 감소시키는 것인데,
이러한 과정이 사라지면 파라미터의 수가 급격히 증가하고 이로인해 더 많은 학습시간을 요구하게 된다.

따라서, Coarse Feature map을 Dense map으로 Upsampling하는 방법을 고려해야 한다.

### Bilinear Interpolation

10x10 이미지를 320x320 이미지로 확대하려면 어떻게 해야할까?

대표적인 방법이 바로 Bilunear interpolation이다.
Bilinear interpolation을 이해하기 위해서는 Linear interpolation을 우선적으로 이해할 필요가 있다.

다음 두 값을 예측해 보자.
![이미지](https://miro.medium.com/max/700/1*GH-pex9uDkeS11wDIJBeTw.png)
위의 값은 5, 아래 값은 7임을 어렵지 않게 예측할 수 있다.
이 처럼 두 지점 사이의 값을 추정할 때 직관적으로 사용하는 방법이 Linear interpolation이다.

위의 추정 방식은 다음과 같이 식으로 표현할 수 있다.
![이미지](https://miro.medium.com/max/700/1*EWqK_U7dJtt456g3_kTUjA.png)
식을 앞의 예제에 대입하면 다음과 같이 추정값을 쉽게 구할 수 있다.
![이미지](https://miro.medium.com/max/700/1*1ItRe38ocUZLKJEkGHUh4Q.png)

Bilinear interpolation은 이러한 1차원의 Linear interpolation을 2차원으로 확장한 것이다.
![이미지](https://miro.medium.com/max/700/1*arKsBkAMZG-EDF2WIVBUNw.png)
네 지점 A,B,C,D 사이의 임의의 점 X를 추정할 수 있다.

이제 우리는 다음과 같은 Feature map의 빈 영역을 추정할 수 있다.
![이미지](https://miro.medium.com/max/700/1*AmGkUff5wN0o7ubdIfc8DQ.png)

### Backwards convolution

Dense prediction을 위한 Upsampling 방법에는 Bilinear interpolation처럼 정해진 방법만 있는 것은 아니다. 즉, Up-sampling도 학습이 가능하다.

Stride가 2이상인 Convolution 연산의 경우 입력 이미지에 대해 크기가 주어든 특징 맵을 출력한다. 이것은 Down-sampling에 해당한다.
![이미지](https://miro.medium.com/max/700/1*hsfu-pv0tyGrN3rHi8njyA.png)

Convolution 연산을 반대로 할 경우 자연스럽게 Up-sampling 효과를 볼 수 있다.
또한, 이 때 사용하는 Filter의 가중치 값은 학습 파라미터에 해당한다.
![이미지](https://miro.medium.com/max/700/1*gtBk1yTapyFzvh00DUkGBA.png)